In [3]:
from pathlib import Path
import numpy as np
import SimpleITK as sitk
from pathlib import Path
from PIL import Image

In [8]:
def read_image(image_filename):
    itk_image = sitk.ReadImage(image_filename)
    npy_image = sitk.GetArrayFromImage(itk_image)
    if npy_image.ndim == 3:
        # 3d, as in original nnunet
        npy_image = npy_image[None]
    elif npy_image.ndim == 4:
        # 4d, multiple modalities in one file
        pass
    else:
        raise RuntimeError(f"Unexpected number of dimensions: {npy_image.ndim} in file {image_filename}")

    return npy_image.astype(np.float32)

def save_image(label, label_folder_path):
    # Create a folder
    label_folder_path.mkdir(exist_ok=True)

    color_pallete = np.array([[0, 0, 0], [255, 0, 0], [0, 255, 0]])

    for i in range(label.shape[1]):
        frame = Image.fromarray(color_pallete[label[0, i].astype(np.uint8)].astype(np.uint8))
        frame.save(label_folder_path / f'{i:05}.png')

In [4]:
dataset_dir = Path('/home/gridsan/nchutisilp/datasets/SAM2_Dataset302_Calcium_OCTv2/')

# test_image_dir = dataset_dir / 'imagesTs'
test_label_dir = dataset_dir / 'Dataset302_Calcium_OCTv2_LaW_Prediction'

# train_image_dir = dataset_dir / 'imagesTr'
train_label_dir = dataset_dir / 'Dataset302_Calcium_OCTv2_LaW_Prediction_imagesTr'

In [9]:
# We have to convert 3D volumes into folders containing image slices
output_dir = Path('/home/gridsan/nchutisilp/datasets/') / 'SAM2_Dataset302_Calcium_OCTv2'
output_dir.mkdir(exist_ok=True)

output_test_label_dir = output_dir / 'LaW_nnUNet_all_predictions_Ts'
output_test_label_dir.mkdir(exist_ok=True)

for volume_path in test_label_dir.glob('*.nii.gz'):
    filename = volume_path.name.replace('.nii.gz', '')

    label_folder_dir = output_test_label_dir / filename
    label_folder_dir.mkdir(exist_ok=True)
    
    label = read_image(volume_path)
    
    save_image(label, label_folder_dir)

In [12]:
# We have to convert 3D volumes into folders containing image slices
output_dir = Path('/home/gridsan/nchutisilp/datasets/') / 'SAM2_Dataset302_Calcium_OCTv2'
output_dir.mkdir(exist_ok=True)

output_train_label_dir = output_dir / 'LaW_nnUNet_all_predictions_Tr'
output_train_label_dir.mkdir(exist_ok=True)

for volume_path in train_label_dir.glob('*.nii.gz'):
    filename = volume_path.name.replace('.nii.gz', '')

    label_folder_dir = output_train_label_dir / filename
    label_folder_dir.mkdir(exist_ok=True)
    
    label = read_image(volume_path)
    
    save_image(label, label_folder_dir)